In [0]:
!git clone https://github.com/clovaai/CRAFT-pytorch.git
!pip install torch==0.4.1 torchvision==0.2.1 opencv-python==3.4.2.17 scikit-image==0.14.2 scipy==1.1.0
!wget https://s3-us-west-1.amazonaws.com/amerrill.files/craft_mlt_25k.pth -P /content/CRAFT-pytorch/
!mkdir /content/CRAFT-pytorch/test-images/

Cloning into 'CRAFT-pytorch'...
remote: Enumerating objects: 53, done.
remote: Total 53 (delta 0), reused 0 (delta 0), pack-reused 53
Unpacking objects: 100% (53/53), done.
--2019-12-03 18:00:38--  https://s3-us-west-1.amazonaws.com/amerrill.files/craft_mlt_25k.pth
Resolving s3-us-west-1.amazonaws.com (s3-us-west-1.amazonaws.com)... 52.219.112.8
Connecting to s3-us-west-1.amazonaws.com (s3-us-west-1.amazonaws.com)|52.219.112.8|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 83152330 (79M) [application/x-www-form-urlencoded]
Saving to: ‘/content/CRAFT-pytorch/craft_mlt_25k.pth’

craft_mlt_25k.pth   100%[===================>]  79.30M  19.2MB/s    in 4.7s    

2019-12-03 18:00:43 (16.9 MB/s) - ‘/content/CRAFT-pytorch/craft_mlt_25k.pth’ saved [83152330/83152330]



## Upload images of Codenames board now




In [0]:
!mv /content/*.jpg /content/CRAFT-pytorch/test-images/
!mv /content/*.jpeg /content/CRAFT-pytorch/test-images/
!mv /content/*.JPG /content/CRAFT-pytorch/test-images/

In [0]:
# from google.colab import files
# uploaded = files.upload()

# import os

# filenames = list(uploaded.keys())
# for fn in filenames:
#   os.environ["image"] = fn
#   !mv /content/$image /content/CRAFT-pytorch/test-images/

## Run text detection model to get bounding boxes around words

In [0]:
!python /content/CRAFT-pytorch/test.py --trained_model=/content/CRAFT-pytorch/craft_mlt_25k.pth --test_folder=/content/CRAFT-pytorch/test-images/

Loading weights from checkpoint (/content/CRAFT-pytorch/craft_mlt_25k.pth)
elapsed time : 19.046854972839355s


## Clone text recognition repo and make directory for cropped images

In [0]:
!git clone https://github.com/clovaai/deep-text-recognition-benchmark.git
!mv /content/deep-text-recognition-benchmark/ /content/dtr
!mkdir /content/dtr/cropped-images/

Cloning into 'deep-text-recognition-benchmark'...
remote: Enumerating objects: 394, done.
remote: Total 394 (delta 0), reused 0 (delta 0), pack-reused 394
Receiving objects: 100% (394/394), 2.99 MiB | 4.69 MiB/s, done.
Resolving deltas: 100% (238/238), done.


In [0]:
import numpy as np

def convert_boxes(boxes):
  """
  Takes in all of the bounding box lists and converts them to dictionaries

  arg1 : list [[top_left_coord, top_right_coord, bottom_right_coord, bottom_left_coord]]
      list of box coordinates

  Returns:
    list of bounding box coordinates as dictionaries [{x1: int, y1: int, x2: int, y2: int}, ...]
  """
  converted = []
  for box in boxes:
    x_values = [box[i] for i in range(0,8,2)]
    y_values = [box[i] for i in range(1,8,2)]
    box_dict = {}
    box_dict['x1'] = min(x_values)
    box_dict['y1'] = min(y_values)
    box_dict['x2'] = max(x_values)
    box_dict['y2'] = max(y_values)
    converted.append(box_dict)
  return converted


def get_word_boxes(boxes):
  """
  Takes in all of the bounding box coordinates and returns the coordinates corresponding to right side up words

  arg1 : list of bounding box coordinates [{x1: int, y1: int, x2: int, y2: int}, ...]
    All bounding boxes

  Returns:
  list of bounding box coordinates [{x1: int, y1: int, x2: int, y2: int}, ...]
    Only boxes that correspond to right side up words
  """
  boxes = sorted(boxes, key=lambda k: k['y1'])
  boxes.reverse()
  filtered_boxes = []
  filtered_boxes.extend(boxes[0:5])
  filtered_boxes.extend(boxes[10:15])
  filtered_boxes.extend(boxes[20:25])
  filtered_boxes.extend(boxes[30:35])
  filtered_boxes.extend(boxes[40:45])
  return filtered_boxes

  # heights = []
  # for i, box in enumerate(boxes):
  #   heights.append({'index': i, 'height': box['y2'] - box['y1']})
  # heights = sorted(heights, key=lambda k: k['height'])
  # print(heights) 
  # med = median([height['height'] for height in heights])
  # def filterLargest(height):
  #   if(height['height'] > med - 1):
  #       return True
  #   else:
  #       return False
  # print(len(heights))
  # filtered_indeces = [height['index'] for height in filter(filterLargest, heights)]
  # filtered_boxes = np.asarray(boxes)[filtered_indeces]
  # print(len(filtered_boxes))
  # assert len(filtered_boxes) == 25
  # return filtered_boxes
  

## Crop words from each image according to bounding boxes

In [0]:
from PIL import Image
from statistics import median
import os

for fn in os.listdir('/content/CRAFT-pytorch/test-images/'):
  img_name_parts = fn.split('.')
  img = Image.open('result/res_' + img_name_parts[0] + '.jpg')
  bnd_box_file = 'result/res_' + img_name_parts[0] + '.txt'
  with open(bnd_box_file) as f:
    boxes = []
    for box in f:
      coords = box.split(',')
      coords = [int(coord) for coord in coords]
      boxes.append(coords)
    boxes = convert_boxes(boxes)
    boxes = get_word_boxes(boxes)
    i = 0
    for box in boxes:
      area = (box['x1'], box['y1'], box['x2'], box['y2'])
      cropped_img = img.crop(area)
      location = '/content/dtr/cropped-images/{}'.format(img_name_parts[0] + '_' + str(i) + '.' + img_name_parts[1])
      cropped_img.save(location)
      i += 1

In [0]:
# !rm /content/deep-text-recognition-benchmark/cropped-images/*

In [0]:
!wget https://s3-us-west-1.amazonaws.com/amerrill.files/TPS-ResNet-BiLSTM-Attn.pth -P /content/dtr/

--2019-12-03 18:05:24--  https://s3-us-west-1.amazonaws.com/amerrill.files/TPS-ResNet-BiLSTM-Attn.pth
Resolving s3-us-west-1.amazonaws.com (s3-us-west-1.amazonaws.com)... 52.219.112.48
Connecting to s3-us-west-1.amazonaws.com (s3-us-west-1.amazonaws.com)|52.219.112.48|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 198678680 (189M) [application/x-www-form-urlencoded]
Saving to: ‘/content/dtr/TPS-ResNet-BiLSTM-Attn.pth’

TPS-ResNet-BiLSTM-A 100%[===================>] 189.47M  20.9MB/s    in 9.9s    

2019-12-03 18:05:34 (19.1 MB/s) - ‘/content/dtr/TPS-ResNet-BiLSTM-Attn.pth’ saved [198678680/198678680]



In [0]:
# !cp results.txt results_bac.txt

cp: cannot stat 'results.txt': No such file or directory


In [0]:
!python /content/dtr/demo.py --Transformation TPS --FeatureExtraction ResNet --SequenceModeling BiLSTM \
                              --Prediction Attn --image_folder /content/dtr/cropped-images/ \
                              --saved_model /content/dtr/TPS-ResNet-BiLSTM-Attn.pth \
                              >> results.txt

In [0]:
# !zip -r /content/cropped-images.zip /content/dtr/cropped-images/

In [0]:
with open('results.txt') as f:
  cur_file = None
  i = 0
  for line in f:
    if line.startswith('/content/dtr/cropped-images/'):
      split = line.split()
      filename = split[0].split('/')[-1].split('_')[0]
      if filename != cur_file:
        cur_file = filename
        i = 0
        print()
        print(filename)
      predicted_word, confidence = split[1], split[2]
      print('{0:<15}'.format(predicted_word), end='')
  
      if (i + 1) % 5 == 0:
        print()
      i += 1
      # for i in range(5):
      #   for j in range(5):
      #     print('{0:<10}'.format(predicted_word), end='')
      #   print()
      # print(predicted_word, confidence, end=', ')

In [0]:
# !rm /content/CRAFT-pytorch/test-images/*
# !rm /content/dtr/cropped-images/*
# !rm /content/result/*

In [0]:
# !unzip /content/cropped-images.zip -o /content/dtr/cropped-images/
!pip3 install fire


     |████████████████████████████████| 81kB 5.6MB/s 
  Created wheel for fire: filename=fire-0.2.1-py2.py3-none-any.whl size=103527 sha256=1272edb38efb5923edd92ed144d995e46800c621c1622667b9e6eeaf93b1e6ed
  Stored in directory: /root/.cache/pip/wheels/31/9c/c0/07b6dc7faf1844bb4688f46b569efe6cafaa2179c95db821da
Successfully built fire


In [0]:
!mkdir -p /content/gt/train/
!mkdir -p /content/gt/val/
!mkdir -p /content/gt/test/

!mkdir /content/dtr/cropped-images/train
!mkdir /content/dtr/cropped-images/val
!mkdir /content/dtr/cropped-images/test

!mkdir -p /content/gt-results/train
!mkdir -p /content/gt-results/val
!mkdir -p /content/gt-results/test

In [0]:
# Split dataset into train, val, test

with open('/content/results.txt') as f:
  with open('/content/gt.txt', 'w') as gt:
    image_results = []
    for line in f:
      if line.startswith('/content/'):
        no_path = line.split('/')[4]
        results = no_path.split()
        image_results.append(results[0] + '\t' + results[1] + '\n')
        # gt.write(results[0] + '\t' + results[1] + '\n')

print(image_results)
print(len(image_results))

['6f4UX_0.jpg\torgan\n', '6f4UX_1.jpg\twar\n', '6f4UX_2.jpg\tstadium\n', '6f4UX_3.jpg\taustralia\n', '6f4UX_4.jpg\ttheater\n', '6f4UX_5.jpg\tray\n', '6f4UX_6.jpg\tlead\n', '6f4UX_7.jpg\tchange\n', '6f4UX_8.jpg\tbermuda\n', '6f4UX_9.jpg\tbeat\n', '6f4UX_10.jpg\tmammoth\n', '6f4UX_11.jpg\tscale\n', '6f4UX_12.jpg\tmillionaire\n', '6f4UX_13.jpg\ttriangle\n', '6f4UX_14.jpg\tleprechaun\n', '6f4UX_15.jpg\thead\n', '6f4UX_16.jpg\tcar\n', '6f4UX_17.jpg\tshadow\n', '6f4UX_18.jpg\tstream\n', '6f4UX_19.jpg\tshakespeare\n', '6f4UX_20.jpg\tcapital\n', '6f4UX_21.jpg\tsnowman\n', '6f4UX_22.jpg\tkangaroo\n', '6f4UX_23.jpg\tjack\n', '6f4UX_24.jpg\tcanada\n', 'BcLvd_0.jpg\tmass\n', 'BcLvd_1.jpg\tpark\n', 'BcLvd_2.jpg\ttap\n', 'BcLvd_3.jpg\talps\n', 'BcLvd_4.jpg\tlawyer\n', 'BcLvd_5.jpg\tstate\n', 'BcLvd_6.jpg\tgermany\n', 'BcLvd_7.jpg\tcrash\n', 'BcLvd_8.jpg\tpaste\n', 'BcLvd_9.jpg\thimalayas\n', 'BcLvd_10.jpg\tpaper\n', 'BcLvd_11.jpg\tindia\n', 'BcLvd_12.jpg\ttrain\n', 'BcLvd_13.jpg\tyard\n', 'BcLvd_14.

In [0]:
train = len(image_results) // (1 / 0.7)
val = (len(image_results) - train) // 2
test = val
# print('Train:',train)
# print('Val:',val)
# print('Test:',test)
results = {'train': [], 'val': [], 'test': []}
assert train + val + test == len(image_results) - 1
for i, result in enumerate(image_results):
  if i <= train:
    results['train'].append(result)
  elif i <= train + val:
    results['val'].append(result)
  else:
    results['test'].append(result) 

for key in results.keys():
  print(key, len(results[key]))

train 627
val 134
test 134


In [0]:
 !export key='train'
 result = '6f4UX_0.jpg\torgan'.split('\t')[0]
 !export result=result
#  !echo $result
 !mv /content/dtr/cropped-images/$result /content/dtr/cropped-images/$key/

In [0]:
for key in results.keys():
  for result in results[key]:
    !export key=key
    filename = result.split('\t')[0]
    !export filename=filename
    !mv /content/dtr/cropped-images/$filename /content/dtr/cropped-images/$key/

In [0]:

!zip -r /content/dtr/cropped-images/train.zip /content/dtr/cropped-images/train
!zip -r /content/dtr/cropped-images/val.zip /content/dtr/cropped-images/val
!zip -r /content/dtr/cropped-images/test.zip /content/dtr/cropped-images/test

In [0]:
# Upload train, val and test zip files, then run this
!unzip /content/train.zip -o /content/dtr/cropped-images/train/
!unzip /content/val.zip -o /content/dtr/cropped-images/val/
!unzip /content/test.zip -o /content/dtr/cropped-images/test/

In [0]:
# Create gt.txt files

for key in results.keys():
  with open('/content/gt/{}/gt.txt'.format(key), 'w') as gt:
    for result in results[key]:
      filename, word = result.split('\t')
      gt.write(result)

In [0]:
!zip -r gt.zip /content/gt

  adding: content/gt/ (stored 0%)
  adding: content/gt/train/ (stored 0%)
  adding: content/gt/train/gt.txt (deflated 73%)
  adding: content/gt/test/ (stored 0%)
  adding: content/gt/test/gt.txt (deflated 73%)
  adding: content/gt/val/ (stored 0%)
  adding: content/gt/val/gt.txt (deflated 71%)


In [0]:
# Training my own text recognition model

!python3 /content/dtr/create_lmdb_dataset.py --inputPath /content/dtr/cropped-images/train --gtFile /content/gt/train/gt.txt --outputPath /content/gt-results/train
!python3 /content/dtr/create_lmdb_dataset.py --inputPath /content/dtr/cropped-images/val --gtFile /content/gt/val/gt.txt --outputPath /content/gt-results/val
!python3 /content/dtr/create_lmdb_dataset.py --inputPath /content/dtr/cropped-images/test --gtFile /content/gt/test/gt.txt --outputPath /content/gt-results/test

Created dataset with 627 samples
Created dataset with 134 samples
Created dataset with 134 samples


In [0]:
!zip -r gt-results.zip /content/gt-results/

  adding: content/gt-results/ (stored 0%)
  adding: content/gt-results/train/ (stored 0%)
  adding: content/gt-results/train/data.mdb (deflated 40%)
  adding: content/gt-results/train/lock.mdb (deflated 99%)
  adding: content/gt-results/test/ (stored 0%)
  adding: content/gt-results/test/data.mdb (deflated 44%)
  adding: content/gt-results/test/lock.mdb (deflated 99%)
  adding: content/gt-results/val/ (stored 0%)
  adding: content/gt-results/val/data.mdb (deflated 44%)
  adding: content/gt-results/val/lock.mdb (deflated 99%)


In [0]:
# Upload train, val and test lmbd datasets then run this

!unzip /content/gt-results.zip -o /content/

Archive:  /content/gt-results.zip
   creating: content/gt-results/
   creating: content/gt-results/train/
  inflating: content/gt-results/train/data.mdb  
  inflating: content/gt-results/train/lock.mdb  
   creating: content/gt-results/test/
  inflating: content/gt-results/test/data.mdb  
  inflating: content/gt-results/test/lock.mdb  
   creating: content/gt-results/val/
  inflating: content/gt-results/val/data.mdb  
  inflating: content/gt-results/val/lock.mdb  


In [0]:
%env CUDA_VISIBLE_DEVICES=0 
!python3 /content/dtr/train.py \
--train_data /content/gt-results/train/ --valid_data /content/gt-results/val/ \
--select_data MJ-ST --batch_ratio 0.5-0.5 \
--Transformation TPS --FeatureExtraction ResNet --SequenceModeling BiLSTM --Prediction Attn

env: CUDA_VISIBLE_DEVICES=0
Filtering the images containing characters which are not in opt.character
Filtering the images whose label is longer than opt.batch_max_length
--------------------------------------------------------------------------------
dataset_root: /content/gt-results/train/
opt.select_data: ['MJ', 'ST']
opt.batch_ratio: ['0.5', '0.5']
--------------------------------------------------------------------------------
dataset_root:    /content/gt-results/train/	 dataset: MJ
Traceback (most recent call last):
  File "/content/dtr/train.py", line 307, in <module>
    train(opt)
  File "/content/dtr/train.py", line 31, in train
    train_dataset = Batch_Balanced_Dataset(opt)
  File "/content/dtr/dataset.py", line 37, in __init__
    _dataset = hierarchical_dataset(root=opt.train_data, opt=opt, select_data=[selected_d])
  File "/content/dtr/dataset.py", line 106, in hierarchical_dataset
    concatenated_dataset = ConcatDataset(dataset_list)
  File "/usr/local/lib/python3.6/di